# 検索機能のsample

In [3]:
%%capture
!pip install qdrant-client
!pip install langchain
!pip install openai
!pip install tiktoken

In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings

from qdrant_client import QdrantClient


In [5]:
model_name = 'text-embedding-ada-002'
model = OpenAIEmbeddings(model = model_name)

In [9]:
# 一度の実行で良い
path = '../tools'
qdrant = QdrantClient(path=path)

RuntimeError: Storage folder ../tools is already accessed by another instance of Qdrant client. If you require concurrent access, use Qdrant server instead.

In [8]:
query = 'All interaction with Qdrant takes place via the REST API.'

hits = qdrant.search(
    collection_name="tool_store",
    query_vector = embeddings.embed_query(query),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'discription': 'Werver mode.qqqqqq', 'code': 'samqqqqqq'} score: 0.7390240945973159
{'discription': 'Werver mode.qqqqqq', 'code': 'samqqqqqq'} score: 0.7390240945973159
{'discription': 'Werver mode.qqqqqq', 'code': 'samqqqqqq'} score: 0.7390240945973159


In [13]:
def run(query: str, model):
    hits = qdrant.search(
        collection_name="tool_store",
        query_vector = embeddings.embed_query(query),
        limit=5
        )
    
    return hits

In [14]:
model_name = 'text-embedding-ada-002'
model = OpenAIEmbeddings(model = model_name)

In [16]:
query = 'All interaction with Qdrant takes place via the REST API.'

search_result = run(query, model)

In [25]:
search_result[0].payload['discription']

'Werver mode.qqqqqq'

In [23]:
search_result[0].payload['code']

'samqqqqqq'

In [27]:
class Searcher():
    def __init__(
        self,
        embedding_model,
        ):
        self.embedding_model = embedding_model

    def run(self, input):
        query = f"Helpful tools to do '{input}'"

        result = qdrant.search(
            collection_name="tool_store",
            query_vector = self.embedding_model.embed_query(query),
            limit=5
            )
            
        return result

In [28]:
input = 'All interaction with Qdrant takes place via the REST API.'
model_name = 'text-embedding-ada-002'
model = OpenAIEmbeddings(model = model_name)


searcher = Searcher(embedding_model = model, input = input)

In [29]:
result = searcher.run()

In [31]:
len(result)

5

In [35]:
search_result = result

executable_tasks = ''

for i in range(len(result)):
    disc = search_result[0].payload['discription']
    code = search_result[0].payload['code']
    executable_tasks += 'discription:' + '\n' + disc +'\n' + 'code:' +'\n' + code + '\n' + '------------------' + '\n'
    
print(executable_tasks)

discription:
Werver mode.qqqqqq
code:
samqqqqqq
------------------
discription:
Werver mode.qqqqqq
code:
samqqqqqq
------------------
discription:
Werver mode.qqqqqq
code:
samqqqqqq
------------------
discription:
Werver mode.qqqqqq
code:
samqqqqqq
------------------
discription:
Werver mode.qqqqqq
code:
samqqqqqq
------------------



## Decider

In [38]:
from langchain.llms import OpenAI

In [44]:
class Decider():
    def __init__(
        self,
        model,
        ):
        self.model = model

    def run(self, input, search_result):
        
        executable_tasks = ''
        
        for i in range(len(search_result)):
            disc = search_result[0].payload['discription']
            code = search_result[0].payload['code']
            
        executable_tasks += 'discription:' + '\n' + disc +'\n' + 'code:' +'\n' + code + '\n' + '------------------' + '\n'

        prompt = """
        You are the agent that determines if the input task is executable.
        Please check the description in the "Task" field and the main processing part of the code to be executed, and decide if the task is executable.
        The task you are to perform is {input_}.
        Answer "Yes." if you can perform the task, or "No." if you cannot.
        Do not output anything other than "Yes." or "No.".

        What you can do is described below.         
        -----------
        {executable_tasks_}
        
        """.format(input_ = input, executable_tasks_ = executable_tasks)

        responese = self.model(prompt)

        if responese == "Yes.":
            print('\033[32m' + 'I do not need to create a tool to run it.\n' + '\033[0m')

        elif responese == "No.":
            print('\033[32m' + 'I must create the tool before executing.\n' + '\033[0m')


        return responese

In [45]:
model = OpenAI(temperature=0,model_name = 'gpt-3.5-turbo')
#model = OpenAI(temperature=0,model_name = 'gpt-4')

decider = Decider(model = model)

input = 'training LightGBM model'

search_result = result

decider.run(input, search_result)

I must create the tool before executing.



'No.'

# 取得したToolを全てappendしたToolのリストを作成する

In [46]:
from langchain.agents import initialize_agent
from langchain.tools import BaseTool

In [47]:
for i in range(len(search_result)):
    disc = search_result[0].payload['discription']
    code = search_result[0].payload['code']

[ScoredPoint(id=6, version=0, score=0.7487178988668953, payload={'discription': 'Werver mode.qqqqqq', 'code': 'samqqqqqq'}, vector=None),
 ScoredPoint(id=5, version=0, score=0.7487178988668953, payload={'discription': 'Werver mode.qqqqqq', 'code': 'samqqqqqq'}, vector=None),
 ScoredPoint(id=4, version=0, score=0.7487178988668953, payload={'discription': 'Werver mode.qqqqqq', 'code': 'samqqqqqq'}, vector=None),
 ScoredPoint(id=3, version=0, score=0.7486261024416454, payload={'discription': 'Werver mode.qqqqqq', 'code': 'samqqqqqq'}, vector=None),
 ScoredPoint(id=1, version=0, score=0.7442231889964104, payload={'discription': 'When you need to scale, simply switch to server mode.', 'code': 'sample'}, vector=None)]

In [ ]:


agent = initialize_agent(tools, model, agent="zero-shot-react-description", verbose=True)

agent.run(input_prompt_)